In [1]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.3 MB/s eta 0:00:00


In [2]:
import wandb
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from joblib import dump, load

In [3]:
# Login to Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
run = wandb.init(entity="flateam", project="the_simpsons_characters", job_type="test")

wandb: Currently logged in as: andre-silva1ifrn (flateam). Use `wandb login --relogin` to force relogin


# Import test data

In [5]:
img_path = run.use_artifact('flateam/the_simpsons_characters/img_data_test:v0', type='split_data').file()
df_img = np.load(img_path)

labels_path = run.use_artifact('flateam/the_simpsons_characters/label_data_test:v0', type='split_data').file()
df_labels = pd.read_csv(labels_path)
df_labels = df_labels['character name']

In [6]:
df_labels.unique()

array(['bart_simpson', 'abraham_grampa_simpson', 'homer_simpson',
       'krusty_the_clown', 'lisa_simpson', 'moe_szyslak',
       'charles_montgomery_burns', 'ned_flanders', 'nelson_muntz',
       'milhouse_van_houten', 'principal_skinner', 'chief_wiggum',
       'marge_simpson', 'mayor_quimby', 'apu_nahasapeemapetilon',
       'ralph_wiggum', 'sideshow_bob', 'lenny_leonard', 'waylon_smithers',
       'miss_hoover', 'barney_gumble', 'edna_krabappel', 'martin_prince',
       'comic_book_guy', 'fat_tony', 'sideshow_mel',
       'groundskeeper_willie', 'maggie_simpson', 'rainier_wolfcastle',
       'selma_bouvier', 'kent_brockman', 'agnes_skinner', 'otto_mann',
       'disco_stu', 'carl_carlson', 'patty_bouvier', 'lionel_hutz', 'gil',
       'snake_jailbird', 'cletus_spuckler', 'professor_john_frink',
       'troy_mcclure'], dtype=object)

# Import trained model

In [ ]:
artifact = run.use_artifact('flateam/the_simpsons_characters/image_processor_class:latest', type='python')
artifact_dir = artifact.download()
file_path = f"{artifact_dir}/myPipeline.py"
!mv {file_path} /content/

wandb:   1 of 1 files downloaded.  


In [ ]:
from myPipeline import ImageProcessor

In [ ]:
artifact = run.use_artifact('flateam/the_simpsons_characters/the_simpsons_character_model_pipeline:v0', type='pipeline')
loaded_pipeline = artifact.file()

pipeline = load(loaded_pipeline)

# Test

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
predictions = pipeline.predict(df_img)

131/131 [==============================] - 11s 83ms/step


In [ ]:
predict = []
for i in predictions:
    predict.append(np.argmax(i))

In [ ]:
label_encoder = LabelEncoder()
df_labels_encoded = label_encoder.fit_transform(df_labels)
print(classification_report(df_labels_encoded, predict))

              precision    recall  f1-score   support

           0       0.91      0.79      0.84       183
           1       0.33      0.12      0.18         8
           2       0.93      0.91      0.92       124
           3       0.20      0.05      0.08        21
           4       0.83      0.72      0.78       269
           5       0.71      0.50      0.59        20
           6       0.82      0.74      0.78       239
           7       0.79      0.91      0.85       197
           8       0.00      0.00      0.00         9
           9       0.78      0.66      0.72        94
          10       0.00      0.00      0.00         2
          11       0.85      0.86      0.85        91
          12       0.00      0.00      0.00         5
          13       0.00      0.00      0.00         5
          14       0.25      0.33      0.29        24
          15       0.74      0.91      0.81       449
          16       0.98      0.85      0.91       100
          17       0.89    

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
